In [370]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models,losses
import matplotlib.pyplot as plt
import numpy as np

In [371]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [372]:
dataset = tf.keras.utils.image_dataset_from_directory(
    'villagepotato',
     shuffle=True,
     image_size=(IMAGE_SIZE, IMAGE_SIZE),
     batch_size= BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [373]:
class_name = dataset.class_names
class_name

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [374]:
for image, label in dataset.take(1):
    print(label)
    print(label.numpy())
    print(image.shape)
    print(image[0].shape)

tf.Tensor([0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 0 1 1 1], shape=(32,), dtype=int32)
[0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 0 1 1 1]
(32, 256, 256, 3)
(256, 256, 3)


In [375]:
# plt.figure(figsize=(10,10))
# for image,label in dataset.take(1):
#     for i in range(12):
#         plt.subplot(3,4,i+1)
#         plt.imshow(image[i].numpy().astype('uint8'))
#         # print(label.numpy())
#         plt.axis('off')
#         plt.title(class_name[label[i].numpy()])

80% => TRAINING
20% => (10% VALIDATION, 10% TEST)

In [376]:
train_size = 0.8
len(dataset)*train_size
train_ds = dataset.take(54)
test_ds = dataset.skip(54)
len(train_ds), len(test_ds)

(54, 14)

In [377]:
val_size = 0.1
len(dataset)*val_size

6.800000000000001

In [378]:
val_ds = test_ds.take(6)
test_ds = test_ds.skip(6)
len(val_ds), len(test_ds)

(6, 8)

In [379]:
def get_dataset_partitions_df(ds, train_split = 0.8, test_split=0.1, valid_split=0.1, shuffle=True, shuffle_size=2000):
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size,seed=12 )
    train_size = int(train_split*ds_size)
    valid_size = int(valid_split*ds_size)

    train_ds = dataset.take(train_size)
    test_ds = dataset.skip(train_size).skip(valid_size)
    valid_ds = dataset.skip(valid_size).take(valid_size)
    return train_ds, valid_ds, test_ds

In [380]:
train_ds, valid_ds, test_ds = get_dataset_partitions_df(dataset)

In [381]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
valid_ds = valid_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

resize and rescale the image

In [382]:
resize_rescale = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1/255)
])

In [383]:
data_augumentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

#### Model training

In [384]:
input_shape = (BATCH_SIZE,IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3
models = tf.keras.models.Sequential([
    resize_rescale,
    data_augumentation,
    tf.keras.layers.Conv2D(32,3,activation='relu', input_shape = input_shape ),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, 3,activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, 3,activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax'),
])
models.build(input_shape=input_shape)

In [385]:
models.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_16 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 sequential_17 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 conv2d_28 (Conv2D)          (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_27 (MaxPooli  (32, 127, 127, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_29 (Conv2D)          (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_28 (MaxPooli  (32, 62, 62, 64)          0         
 ng2D)                                               

#### model compile

In [391]:
models.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

#### model fit

In [395]:
history = models.fit(
    train_ds,
    epochs=15,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=valid_ds
)

Epoch 1/15


54/54 [==============================] - 92s 2s/step - loss: 0.1622 - accuracy: 0.9381 - val_loss: 0.2368 - val_accuracy: 0.8958
Epoch 2/15
54/54 [==============================] - 90s 2s/step - loss: 0.1250 - accuracy: 0.9485 - val_loss: 0.4790 - val_accuracy: 0.8698
Epoch 3/15
54/54 [==============================] - 87s 2s/step - loss: 0.1345 - accuracy: 0.9468 - val_loss: 0.5583 - val_accuracy: 0.8438
Epoch 4/15
54/54 [==============================] - 94s 2s/step - loss: 0.1143 - accuracy: 0.9560 - val_loss: 0.3275 - val_accuracy: 0.8594
Epoch 5/15
54/54 [==============================] - 88s 2s/step - loss: 0.1073 - accuracy: 0.9630 - val_loss: 0.3848 - val_accuracy: 0.8594
Epoch 6/15
54/54 [==============================] - 82s 2s/step - loss: 0.0740 - accuracy: 0.9734 - val_loss: 0.5066 - val_accuracy: 0.8490
Epoch 7/15
54/54 [==============================] - 82s 2s/step - loss: 0.0874 - accuracy: 0.9699 - val_loss: 0.1303 - val_accuracy: 0.9375
Epoch 8/15
54/54 [=============